## 🧩 Importation des bibliothèques principales

Cette première cellule regroupe l’ensemble des **modules nécessaires** pour la création et la gestion du dataset.  
Chaque bibliothèque joue un rôle précis :

- **os** → gestion des fichiers et dossiers  
- **cv2 (OpenCV)** → traitement et transformation des images  
- **numpy (np)** → manipulation de tableaux et opérations matricielles  
- **pandas (pd)** → création et gestion des DataFrames (données tabulaires)  
- **itertools.product** → parcourir efficacement toutes les tailles d’images sans multiplier les boucles.  
- **tqdm** → affichage de barres de progression dans les boucles

> ⚙️ Cette étape initialise l’environnement Python nécessaire avant le chargement et le traitement des images.


In [15]:
import os
import cv2
import numpy as np
import pandas as pd
from itertools import product
from tqdm import tqdm

## 🏗️ Génération du Dataset à différentes tailles d’images

Dans cette étape, nous allons **construire le dataset complet** à partir des images brutes des quatre classes disponibles.  
L’objectif est de créer plusieurs versions du dataset selon différentes **tailles d’images** (64×64, 128×128, etc.) pour comparer les performances des modèles en fonction de la résolution.

---

### ⚙️ Étapes du traitement

1. **Initialisation des structures de stockage**
   - `X_dict` : contient les *features* (les images transformées et aplaties)
   - `y_dict` : contient les *labels* (la classe associée à chaque image)
   - `df_dict` : contiendra les DataFrames finaux avant la sauvegarde

2. **Définition des classes et équilibre**
   - Les catégories utilisées sont :  
     - `0 → Normal`  
     - `1 → COVID`  
     - `2 → Autres pathologies (Lung Opacity et Viral Pneumonia)`  
   - Pour éviter tout **déséquilibre entre les classes** :  
     - On choisit **le même nombre d’images pour Normal et COVID**.  
     - Les images de Lung Opacity et Viral Pneumonia sont regroupées dans la classe `2`, avec un nombre d’images total équivalent aux autres classes.  
   - Résultat : **toutes les classes ont le même nombre d’images**, garantissant un dataset équilibré.
     

3. **Préparation et allocation mémoire**
   - Allocation de tableaux `NumPy` adaptés à la taille d’image et au nombre total d’échantillons.
   - Permet d’éviter les ralentissements dus à la création dynamique de tableaux pendant la boucle.

4. **Boucle principale**
   - Pour chaque **taille d’image** :  
     - Lecture et redimensionnement des images et masques (`cv2.resize`)  
     - Application d’un **masquage** (`cv2.bitwise_and`) pour ne conserver que la zone utile  
     - Aplatissement des images pour les stocker sous forme de vecteurs  
     - Enregistrement des labels correspondants

5. **Création du DataFrame final**
   - Conversion des tableaux `X` et `y` en un `DataFrame Pandas`  
   - Mélange aléatoire des lignes (`sample(frac=1)`) pour éviter tout biais d’ordre  
   - Sauvegarde du dataset en **format HDF5** (`to_hdf`) pour une lecture rapide et efficace lors de l’entraînement

---

### 💾 Sorties générées

Pour chaque taille d’image, un fichier est produit, avec **un nombre équilibré d’images par classe**.  
Cela garantit que les modèles ne seront pas influencés par une sur-représentation d’une classe particulière, ce qui est essentiel pour un apprentissage fiable et représentatif.


In [17]:
# === Initialisation ===
X_dict = {}   # Dictionnaire qui contiendra les images (features)
y_dict = {}   # Dictionnaire qui contiendra les labels
df_dict = {}  # Dictionnaire pour stocker les DataFrames finaux




# Nous allons generer les datasets du jeu avec les différentes tailles
# === Remarque ===
# Nous allons garder des images pour le test via Streamlit 
# qui n'ont pas été vues par le modèle.
# Réservation : 
# - 6 images pour les classes Normal et COVID
# - 3 images pour les classes Lung_Opacity et Viral Pneumonia
classes = {
          0: ["Normal/images", "Normal/masks",3610,0],
          1: ["COVID/images", "COVID/masks",3610,1],
          2: ["Lung_Opacity/images", "Lung_Opacity/masks",2268,2],
          3: ["Viral Pneumonia/images", "Viral Pneumonia/masks",1342,2],
}

Nb=10830 # Nombre total d'images (somme des sous-ensembles)
tailles = [64,128,176,224,299]  # Différentes tailles à générer

# Pré-allocation mémoire
X_dict[Nb]={}
y_dict[Nb]={}
df_dict[Nb]={}


for size in tailles:
    X_dict[Nb][size] = np.zeros((Nb, size * size), dtype=np.uint8)
    y_dict[Nb][size] = np.zeros((Nb,), dtype=np.uint8)

	
for lot, taille in product([10830], tailles):
    index_global = 0
    print(f"\n=== Traitement des images pour taille {taille}x{taille} ===")
    
    for indice ,(img,mask,num,classe) in classes.items():
        contenu_image=os.listdir(img)
        contenu_mask=os.listdir(mask)
        for i,(imag,masks) in tqdm(enumerate(zip(contenu_image[:num],contenu_mask[:num]))):
            # Lecture et redimension
            img_covid4=cv2.imread(os.path.join(img,imag),cv2.IMREAD_GRAYSCALE)
            mask_covid4=cv2.imread(os.path.join(mask,masks),cv2.IMREAD_GRAYSCALE)
            
            if img_covid4 is None or mask_covid4 is None:
                continue  # sécurité en cas d’image corrompue
                
            img_resize=cv2.resize(img_covid4,dsize=(taille,taille))
            mask_resize=cv2.resize(mask_covid4,dsize=(taille,taille))
         
            # Fusion image + masque
            melange_covid4=cv2.bitwise_and(img_resize,mask_resize)
            
            
            # Données avec les images aplatie dans X et cible dans y
            X_dict[lot][taille][index_global,:] = melange_covid4.reshape(-1)
            y_dict[lot][taille][index_global]=classe
            index_global += 1
                
    # Création DataFrame final
    df=pd.DataFrame(X_dict[lot][taille])
    df["Cible"]=y_dict[lot][taille]
    df.columns =df.columns.map(str)
    
    
    # Mélange aléatoire
    df=df.sample(frac=1,random_state=42).reset_index(drop=True)
    
    
    # Sauvegarde en HDF5
    filename = f"dataset_{taille}_{Nb}.h5"
    df.to_hdf(filename, key=f"df_{taille}_{Nb}", mode="w", format="fixed")




=== Traitement des images pour taille 64x64 ===


3610it [00:14, 249.87it/s]
3610it [00:13, 268.12it/s]
2268it [00:08, 275.39it/s]
1342it [00:05, 242.71it/s]



=== Traitement des images pour taille 128x128 ===


3610it [00:13, 264.24it/s]
3610it [00:13, 271.90it/s]
2268it [00:07, 291.85it/s]
1342it [00:05, 264.50it/s]



=== Traitement des images pour taille 176x176 ===


3610it [00:13, 267.17it/s]
3610it [00:13, 261.19it/s]
2268it [00:08, 267.24it/s]
1342it [00:05, 254.81it/s]



=== Traitement des images pour taille 224x224 ===


3610it [00:15, 240.17it/s]
3610it [00:15, 238.76it/s]
2268it [00:09, 251.83it/s]
1342it [00:06, 202.53it/s]



=== Traitement des images pour taille 299x299 ===


3610it [00:15, 238.28it/s]
3610it [00:15, 225.87it/s]
2268it [00:09, 231.72it/s]
1342it [00:06, 223.04it/s]


## 📂 Chargement et vérification des datasets HDF5

Une fois les différents datasets créés et sauvegardés au format **HDF5**, nous les rechargeons ici pour vérifier leur **intégrité** et **équilibre des classes**.

---

### ⚙️ Étapes effectuées

1. **Chargement des fichiers**
   - Pour chaque taille d’image (64×64, 128×128, 176×176, 224×224, 299×299), le fichier correspondant est chargé depuis le disque :
     ```
     dataset_<taille>_10830.h5
     ```
   - Le contenu est lu via la clé :
     ```
     df_<taille>_10830
     ```
   - Chaque DataFrame est stocké dans le dictionnaire `dfs` pour un accès facile selon la taille.

2. **Vérification de la répartition des classes**
   - Pour chaque DataFrame, la distribution de la colonne **`Cible`** (les labels de classes) est affichée.
   - Cela permet de confirmer :
     - qu’aucune image n’a été perdue pendant la génération,
     - que les classes sont bien équilibrées (ou de repérer un éventuel déséquilibre).

---

### 🧾 Exemple de sortie attendue



In [32]:
# === Lecture du Dataframe depuis HDF5 ===
dfs={}
tailles = [64,128,176,224,299]
for t in tailles:
    filename=f"dataset_{t}_10830.h5"
    key=f"df_{t}_10830"
    dfs[t]=pd.read_hdf(filename,key=key)
    
for t in tailles:
    print(f"====vérification de l'equilibre des classes pour la taille {t}x{t}:")
    print(dfs[t]["Cible"].value_counts())


====vérification de l'equilibre des classes pour la taille 64x64:
Cible
0    3610
2    3610
1    3610
Name: count, dtype: int64
====vérification de l'equilibre des classes pour la taille 128x128:
Cible
0    3610
2    3610
1    3610
Name: count, dtype: int64
====vérification de l'equilibre des classes pour la taille 176x176:
Cible
0    3610
2    3610
1    3610
Name: count, dtype: int64
====vérification de l'equilibre des classes pour la taille 224x224:
Cible
0    3610
2    3610
1    3610
Name: count, dtype: int64
====vérification de l'equilibre des classes pour la taille 299x299:
Cible
0    3610
2    3610
1    3610
Name: count, dtype: int64


## 🧮 Vérification des dimensions des DataFrames

Après avoir confirmé l’équilibre des classes, il est essentiel de vérifier la **structure** de chaque DataFrame pour s’assurer que :

- le **nombre total de lignes** correspond bien au total d’images attendu,  
- le **nombre de colonnes** (features extraites + colonne cible) est cohérent entre les différentes tailles d’images.

---

### ⚙️ Étapes effectuées

Pour chaque taille d’image (`64x64`, `128x128`, `176x176`, `224x224`, `299x299`) :
- On affiche le résultat de la méthode `shape`, qui retourne :


In [36]:
for t in tailles:
    print(f"====vérification des lignes et colonnes pour chaques classes pour la taille {t}x{t}:")
    print(dfs[t].shape)
    

====vérification des lignes et colonnes pour chaques classes pour la taille 64x64:
(10830, 4097)
====vérification des lignes et colonnes pour chaques classes pour la taille 128x128:
(10830, 16385)
====vérification des lignes et colonnes pour chaques classes pour la taille 176x176:
(10830, 30977)
====vérification des lignes et colonnes pour chaques classes pour la taille 224x224:
(10830, 50177)
====vérification des lignes et colonnes pour chaques classes pour la taille 299x299:
(10830, 89402)


## ✅ Conclusion : Validation des jeux de données

Les différents datasets générés présentent les caractéristiques suivantes :

- **Cohérence structurelle :**  
  Chaque DataFrame possède le bon nombre de lignes et de colonnes selon la taille d’image choisie.

- **Équilibre des classes :**  
  Les classes sont réparties de manière homogène, garantissant une bonne représentativité lors de l’entraînement.

- **Disponibilité multi-tailles :**  
  Les jeux de données sont disponibles pour plusieurs résolutions (`64x64`, `128x128`, `176x176`, `224x224`, `299x299`), ce qui permettra de comparer les performances des modèles selon la taille des images.

---

💡 **Conclusion finale :**  
Ces datasets sont **cohérents**, **équilibrés** et **prêts pour le prétraitement** ainsi que pour **l’apprentissage des modèles**.  
Si un ralentissement se fait sentir, nous pourrons **réduire le nombre d’images** utilisées pour accélérer les expérimentations sans compromettre la validité des résultats.
